# Data Organization

## Prepare protein embedding

Get the embedding.pt file for each species's genes, each species only needs to be done once."

In [59]:
import torch
from tqdm import tqdm

In [ ]:
pep_file = 'data/pep.fa' # 输入蛋白序列，确保每个基因只有一个
tmp_dir = '/path/to/pep_tmp' # 临时目录

In [ ]:
!conda run -n SATURN python extract.py esm2_t48_15B_UR50D  $pep_file $tmp_dir --repr_layers 0 47 48 --include mean per_tok &> run.log &

In [38]:
import pickle
with open('./data/sm_pt.pkl', 'rb') as f:
    sm_pt = pickle.load(f)

In [ ]:

# 水稻
# # key是基因，value是一组转录本
# gene_symbol_to_protein_ids = {}
# for transcript_id in os_tx_id:
#     gene_id = transcript_id.split("-")[0].replace("t", "g")
#     if gene_id in gene_symbol_to_protein_ids:
#         gene_symbol_to_protein_ids[gene_id].append(transcript_id)
#     else:
#         gene_symbol_to_protein_ids[gene_id] = [transcript_id]

# # Average the embeddings of all transcripts of the same gene

# # Average protein embeddings for each gene to get gene embeddings
# os_protein_id_to_embedding = {
#     gene_symbol: torch.mean(
#         torch.stack([
#             protein_id_to_embedding[protein_id] for protein_id in protein_ids
#         ]),
#         dim=0
#     )
#     for gene_symbol, protein_ids in tqdm(gene_symbol_to_protein_ids.items())
# }



In [44]:
# # Trinity的编号是 _ , 需要替换成 -

# protein_id_to_embedding = sm_pt

# # key是基因，value是一组转录本
# gene_symbol_to_protein_ids = {}
# for transcript_id in protein_id_to_embedding.keys():
#     gene_id = transcript_id.replace("_", "-")
#     if gene_id in gene_symbol_to_protein_ids:
#         gene_symbol_to_protein_ids[gene_id].append(transcript_id)
#     else:
#         gene_symbol_to_protein_ids[gene_id] = [transcript_id]

# # Average protein embeddings for each gene to get gene embeddings

# protein_id_to_embedding_avg = {
#     gene_symbol: torch.mean(
#         torch.stack([
#             protein_id_to_embedding[protein_id] for protein_id in protein_ids
#         ]),
#         dim=0
#     )
#     for gene_symbol, protein_ids in tqdm(gene_symbol_to_protein_ids.items())
# }


In [47]:
# # Save gene symbol to embedding map
# torch.save(protein_id_to_embedding_avg, "data/sm_protein_id_to_embedding.pt")

# 保存最后的结果

In [56]:
# Make the csv
import pandas as pd

df = pd.DataFrame(columns=["path", "species", "embedding_path"])
df["species"] = ["at", "ne", 'os', 'pt', 'ss', 'sm']
df["path"] = [
"/data/xzg_data/shoot-cross-species-integration/input/at.h5ad",
"/data/xzg_data/shoot-cross-species-integration/input/ne.h5ad",
"/data/xzg_data/shoot-cross-species-integration/input/os.h5ad",
"/data/xzg_data/shoot-cross-species-integration/input/pt.h5ad",
"/data/xzg_data/shoot-cross-species-integration/input/ss_v2.h5ad",
"/data/xzg_data/shoot-cross-species-integration/input/sm.h5ad"
    ]

##############################
df["embedding_path"] = [
"/data/xzg_data/shoot-cross-species-integration/input/at_protein_id_to_embedding.pt",
"/data/xzg_data/shoot-cross-species-integration/input/ne_protein_id_to_embedding.pt",
"/data/xzg_data/shoot-cross-species-integration/input/os_protein_id_to_embedding.pt",
"/data/xzg_data/shoot-cross-species-integration/input/pt_protein_id_to_embedding.pt",
"/data/xzg_data/shoot-cross-species-integration/input/ss_protein_id_to_embedding.pt",
"/data/xzg_data/shoot-cross-species-integration/input/sm_protein_id_to_embedding.pt"
]
df.to_csv("input/at_ne_os_pt_ss_sm.csv", index=False)
df

,path,species,embedding_path
0,/data/xzg_data/shoot-cross-species-integration...,at,/data/xzg_data/shoot-cross-species-integration...
1,/data/xzg_data/shoot-cross-species-integration...,ne,/data/xzg_data/shoot-cross-species-integration...
2,/data/xzg_data/shoot-cross-species-integration...,os,/data/xzg_data/shoot-cross-species-integration...
3,/data/xzg_data/shoot-cross-species-integration...,pt,/data/xzg_data/shoot-cross-species-integration...
4,/data/xzg_data/shoot-cross-species-integration...,ss,/data/xzg_data/shoot-cross-species-integration...
5,/data/xzg_data/shoot-cross-species-integration...,sm,/data/xzg_data/shoot-cross-species-integration...


In [57]:
import os
full_path = os.path.abspath("input/at_ne_os_pt_ss_sm.csv")

In [58]:
!cd ~/software/SATURN && conda run -n SATURN python3 ./train-saturn.py \
    --in_data={full_path} --in_label_col="seurat_high" \
        --ref_label_col="seurat_high" \
    --embedding_model="ESM2" --batch_size=10240 --epochs=200 

Using Device 0
Set seed to 0
After loading the anndata at View of AnnData object with n_obs × n_vars = 27812 × 22525
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'old.ident', 'seurat_clusters', 'seurat_layer', 'seurat_clusters_1', 'seurat_high', 'species', 'species_type_label', 'truth_labels', 'ref_labels'
After loading the anndata ne View of AnnData object with n_obs × n_vars = 13557 × 19991
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'RNA_snn_res.0.5', 'seurat_clusters', 'RNA_snn_res.1.4', 'RNA_snn_res.0.7', 'RNA_snn_res.1', 'RNA_snn_res.0.4', 'RNA_snn_res.0.6', 'RNA_snn_res.1.7', 'seurat_clusters_1', 'seurat_layer', 'RNA_snn_res.1.1', 'RNA_snn_res.2.4', 'RNA_snn_res.2.5', 'RNA_snn_res.5', 'scales_score', 'seurat_high', 'seurat_clusters_2', 'species', 'species_type_label', 'truth_labels', 'ref_labels'
After loading the anndata os View of AnnData object with n_obs × n_vars = 20959 × 35687
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'RNA_snn_res.0.5', 'seurat_

In [61]:
# 将分析结果移动到当前目录下
!mv ~/software/SATURN/out/saturn_results/ .

In [ ]:
# 在SATURN目录下运行
# !python3 ./train-saturn.py --in_data=data/frog_zebrafish_run.csv \
#                               --in_label_col=cell_type --ref_label_col=cell_type \
#                               --num_macrogenes=2000     --hv_genes=8000          \
#                               --centroids_init_path=saturn_results/fz_centroids.pkl \
#                               --score_adata --ct_map_path=data/frog_zebrafish_cell_type_map.csv \
#                               --work_dir=. \
#                               --device_num=7 \